<div align=center> <font color="#3C5BDA" size=8 face="微软雅黑"> Arrival Time Prediction using a Stacked Model </font> </div>

### Table of Contents:
- 1. Import Dependencies
- 2. Exploratory Data Analysis (EDA) and Data Preprocessing
    - Loading and Processing flight_data.csv and JFK NYC LGA Weather.csv
    - Creating, Loading and Processing JFK EWR and LGA Flights.csv
- 3. Data Modeling and Evaluation
    - Creating a XGBoost Regressor
    - Creating a CatBoost Regressor
    - Evaluating the CatBoost and XGBoost Regressor
    - Creating a Stacked Model
    - Evaluating the Stacked Model
- 4. Conclusion

<div align=center> <font color="#3C5BDA" size=6 face="1. Import Dependencies"> Arrival Time Prediction using a Stacked Model </font> </div>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import tensorflow as tf
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.layers import Dropout, Input, Dense
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import StackingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from catboost import CatBoostRegressor
import xgboost as xgb 
from itertools import product
import regex as re
import optuna
import holidays
from pickle import dump, load
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'tensorflow'